In [859]:
import clingo

from starlingo.Atom import Atom
from starlingo.Literal import BasicLiteral
from starlingo.Program import from_string, Program, evaluate_forwards
from starlingo.Rule import NormalRule, Constraint, Fact
from starlingo.Symbol import Function, IntegerConstant, Term

In [860]:

effect_str = """

effect(at(R,P2),move(R,P2),(at(R,P1),),(-at(_,P2),)) :- robot(R), adjacent(P1, P2).


"""
effect_raw = from_string(effect_str)[0]
print(effect_raw.custom_str(sep='\n'))

#program base.
effect(at(R,P2),move(R,P2),(at(R,P1)),(-at(_,P2))) :- robot(R), adjacent(P1,P2).


In [861]:
effect_debug = """

#program effect(__t).
at(R,P2,__t+1) :- robot(R), adjacent(P1,P2), move(R,P2,__t).
:- robot(R), adjacent(P1,P2), move(R,P2,__t), not at(R,P1,__t).
:- robot(R), adjacent(P1,P2), move(R,P2,__t), at(_,P2,__t).

"""

In [862]:
__t = Function('__t')
one = Term(IntegerConstant(1))

In [863]:
effect_rules = []
for effect_rule in effect_raw.rules:
    if effect_rule.is_rule() and not effect_rule.is_constraint():
        if effect_rule.is_normal_rule():
            symbol = effect_rule.head.atom.symbol
            if symbol.name == 'effect':
                effect = BasicLiteral(Atom(symbol.arguments[0].arguments_append(__t + one)))
                action = BasicLiteral(Atom(symbol.arguments[1].arguments_append(__t)))
                conditions = tuple(
                    BasicLiteral(Atom(condition.arguments_append(__t))).strong_neg_to_default_neg() for condition in
                    symbol.arguments[2].arguments)
                preconditions = (-BasicLiteral(Atom(condition.arguments_append(__t))).strong_neg_to_default_neg() for
                                 condition in symbol.arguments[3].arguments)

                effect_rule_head = effect
                effect_rule_body = (*effect_rule.body, action, *conditions)
                effect_rule = NormalRule(effect_rule_head, effect_rule_body)
                effect_rules.append(effect_rule)

                for precondition in preconditions:
                    precondition_constraint_body = (*effect_rule.body, action, *conditions, precondition)
                    precondition_constraint = Constraint(precondition_constraint_body)
                    effect_rules.append(precondition_constraint)

effect_p = Program(name='effect', parameters=(__t,), rules=effect_rules)
print(effect_p.custom_str(sep='\n'))

#program effect(__t).
at(R,P2,__t+1) :- robot(R), adjacent(P1,P2), move(R,P2,__t), at(R,P1,__t).
:- robot(R), adjacent(P1,P2), move(R,P2,__t), at(R,P1,__t), move(R,P2,__t), at(R,P1,__t), at(_,P2,__t).


In [864]:
domain_str = """

#program domain.

row(1). col(1).
row(2). col(2).
row(3). col(3).
loc((X,Y)) :- col(X), row(Y), (X,Y) != (1,3), (X,Y) != (3,1).

adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X,Y+1).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X+1,Y).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X,Y-1).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X-1,Y).

1 { at(R,P,0) : loc(P) } 1 :- robot(R).

#program domain(__t).

at(R,P,__t+1) :- at(R,P,__t), not move(R,_,__t).

:- { at(R,P,__t) : loc(P) } <= 0, robot(R).
%:- 2 <= { at(R,P,__t) : loc(P) }, robot(R).
:- 2 <= { at(R,P,__t) : robot(R) }, loc(P).

"""
print(domain_str)



#program domain.

row(1). col(1).
row(2). col(2).
row(3). col(3).
loc((X,Y)) :- col(X), row(Y), (X,Y) != (1,3), (X,Y) != (3,1).

adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X,Y+1).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X+1,Y).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X,Y-1).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X-1,Y).

1 { at(R,P,0) : loc(P) } 1 :- robot(R).

#program domain(__t).

at(R,P,__t+1) :- at(R,P,__t), not move(R,_,__t).

:- { at(R,P,__t) : loc(P) } <= 0, robot(R).
%:- 2 <= { at(R,P,__t) : loc(P) }, robot(R).
:- 2 <= { at(R,P,__t) : robot(R) }, loc(P).




In [865]:
plan_str = """

#program base.

#show move/3.
#show at/3.

#program plan(__t).

0 { move(R,P,__t) : loc(P) } 1 :- robot(R).

#program init.

:- not at(worker, (1,1), 0).
:- not at(charger, (2,2), 0).

#program goal(__t).

:- not at(worker, (3,3), __t).
:- not at(charger, (2,2), __t).

"""
print(plan_str)



#program base.

#show move/3.
#show at/3.

#program plan(__t).

0 { move(R,P,__t) : loc(P) } 1 :- robot(R).

#program init.

:- not at(worker, (1,1), 0).
:- not at(charger, (2,2), 0).

#program goal(__t).

:- not at(worker, (3,3), __t).
:- not at(charger, (2,2), __t).




In [866]:
instance_str = """

#program base.

robot(worker).
robot(charger).

"""
print(instance_str)



#program base.

robot(worker).
robot(charger).




In [867]:
plans = tuple(evaluate_forwards((effect_p, domain_str, plan_str, instance_str), parts=(
    ('base', ()),
    ('domain', ()),
    ('init', ()),
    ('goal', (clingo.Number(4),)),
    *(('effect', (clingo.Number(n),)) for n in range(0, 4)),
    *(('domain', (clingo.Number(n),)) for n in range(0, 4)),
    *(('plan', (clingo.Number(n),)) for n in range(0, 4)),
), report=True));

Answer 1: {
at(charger,(2,2),0)
at(charger,(2,2),4)
at(charger,(2,3),1)
at(charger,(2,3),2)
at(charger,(2,3),3)
at(worker,(1,1),0)
at(worker,(2,1),1)
at(worker,(2,2),2)
at(worker,(3,2),3)
at(worker,(3,3),4)
move(charger,(2,2),3)
move(charger,(2,3),0)
move(worker,(2,1),0)
move(worker,(2,2),1)
move(worker,(3,2),2)
move(worker,(3,3),3)
}
Answer 2: {
at(charger,(1,2),1)
at(charger,(1,2),2)
at(charger,(1,2),3)
at(charger,(2,2),0)
at(charger,(2,2),4)
at(worker,(1,1),0)
at(worker,(2,1),1)
at(worker,(2,2),2)
at(worker,(3,2),3)
at(worker,(3,3),4)
move(charger,(1,2),0)
move(charger,(2,2),3)
move(worker,(2,1),0)
move(worker,(2,2),1)
move(worker,(3,2),2)
move(worker,(3,3),3)
}
Answer 3: {
at(charger,(2,2),0)
at(charger,(2,2),4)
at(charger,(2,3),3)
at(charger,(3,2),1)
at(charger,(3,3),2)
at(worker,(1,1),0)
at(worker,(2,1),1)
at(worker,(2,2),2)
at(worker,(3,2),3)
at(worker,(3,3),4)
move(charger,(2,2),3)
move(charger,(2,3),2)
move(charger,(3,2),0)
move(charger,(3,3),1)
move(worker,(2,1),0)
move(worke

In [868]:
plan = sorted(plans[0], key=lambda atom: atom.symbol.arguments[-1])

In [869]:
plan_diagnosis = Program(rules=tuple(Fact(atom) for atom in plan))

In [870]:
observed_str = """

at(charger,(2,2),0).
at(charger,(2,2),1).
at(charger,(2,2),2).
at(charger,(2,2),3).
at(charger,(2,2),4).
at(worker,(1,1),0).
at(worker,(2,1),1).
at(worker,(2,1),2).
at(worker,(2,1),3).
at(worker,(2,1),4).

"""

observed = sorted(next(evaluate_forwards((observed_str,))), key=lambda atom: atom.symbol.arguments[-1])

In [871]:
discrepancy_str = """

discrepancy(T,((R,P,T),(R,Q,T))) :- at(R,P,T), at(R,Q,T), Q>P.
first_discrepancy(T) :- T=#min {D : discrepancy(D,_) }.

:- not discrepancy(_,_).

#show first_discrepancy/1.
#show discrepancy/2.
"""
discrepancy = tuple(evaluate_forwards((plan_diagnosis, observed_str, discrepancy_str), report=True))[0]

Answer 1: {
first_discrepancy(1)
discrepancy(1,((charger,(2,2),1),(charger,(2,3),1)))
discrepancy(2,((charger,(2,2),2),(charger,(2,3),2)))
discrepancy(2,((worker,(2,1),2),(worker,(2,2),2)))
discrepancy(3,((charger,(2,2),3),(charger,(2,3),3)))
discrepancy(3,((worker,(2,1),3),(worker,(3,2),3)))
discrepancy(4,((worker,(2,1),4),(worker,(3,3),4)))
}
SAT 1


In [872]:
first_discrepancy = next((atom for atom in discrepancy if atom.symbol.match('first_discrepancy', 1))).symbol.arguments[
    0]
print(first_discrepancy)

1


In [873]:
expected_rules = []

last_known_time = -1
for step in plan:
    if step.symbol.match('at', 3):
        if step.symbol.arguments[-1] == first_discrepancy:
            expected_constraint = Constraint((-BasicLiteral(Atom(step.symbol)),))
            expected_rules.append(expected_constraint)
    elif step.symbol.match('move', 3):
        if step.symbol.arguments[-1] < first_discrepancy:
            expected_constraint = Constraint((-BasicLiteral(Atom(step.symbol)),))
            expected_rules.append(expected_constraint)

expected = Program(name='expected', rules=expected_rules)
print(expected.custom_str(sep='\n'))

#program expected.
:- not move(charger,(2,3),0).
:- not move(worker,(2,1),0).
:- not at(charger,(2,3),1).
:- not at(worker,(2,1),1).


In [874]:
robot_components_str = """

#program base.

robot_component(R,base) :- robot(R).

"""
print(robot_components_str)



#program base.

robot_component(R,base) :- robot(R).




In [875]:
disables_str = """

disables(((R,base),),move(R,P2),(at(R,P1),)) :- robot(R), adjacent(P1,P2).

"""
disables_p = from_string(disables_str)[0]

In [876]:
broken_str = """

#program broken.

{ broken(R,P,__t) } :- robot_component(R,P).

#program broken(__t).

-broken(R,P,__t) :- not broken(R,P,__t), robot_component(R,P).
broken(R,P,__t+1) :- not -broken(R,P,__t+1), -broken(R,P,__t), robot_component(R,P).
broken(R,P,__t+1) :- broken(R,P,__t).

"""
print(broken_str)



#program broken.

{ broken(R,P,__t) } :- robot_component(R,P).

#program broken(__t).

-broken(R,P,__t) :- not broken(R,P,__t), robot_component(R,P).
broken(R,P,__t+1) :- not -broken(R,P,__t+1), -broken(R,P,__t), robot_component(R,P).
broken(R,P,__t+1) :- broken(R,P,__t).




In [877]:
diagnosis_str = """

#program diagnosis(__t).

{ diagnosis(R,P,__t) : robot_component(R,P) }.

:- not broken(R,P, __t), diagnosis(R,P,__t).
:- not -broken(R,P, __t), robot_component(R,P), not diagnosis(R,P,__t).

"""
print(diagnosis_str)



#program diagnosis(__t).

{ diagnosis(R,P,__t) : robot_component(R,P) }.

:- not broken(R,P, __t), diagnosis(R,P,__t).
:- not -broken(R,P, __t), robot_component(R,P), not diagnosis(R,P,__t).




In [878]:
diagnostic_domain_rules = []
for effect_rule in effect_raw.rules:
    if effect_rule.is_rule() and not effect_rule.is_constraint():
        if effect_rule.is_normal_rule():
            symbol = effect_rule.head.atom.symbol
            if symbol.name == 'effect':
                effect = BasicLiteral(Atom(symbol.arguments[0].arguments_append(__t + one)))
                action_ = symbol.arguments[1]
                action = BasicLiteral(Atom(action_.arguments_append(__t)))
                conditions = tuple(
                    BasicLiteral(Atom(condition.arguments_append(__t))).strong_neg_to_default_neg() for condition in
                    symbol.arguments[2].arguments)
                preconditions = tuple(
                    -BasicLiteral(Atom(condition.arguments_append(__t))).strong_neg_to_default_neg() for
                    condition in symbol.arguments[3].arguments)
                precondition_1_head = BasicLiteral(Atom(Function('pre', (action_, __t))))
                precondition_1_body = (*effect_rule.body, -BasicLiteral(Atom(-Function('pre', (action_, __t)))),)
                precondition_1 = NormalRule(precondition_1_head, precondition_1_body)
                diagnostic_domain_rules.append(precondition_1)

                for precondition in preconditions:
                    precondition_2_head = BasicLiteral(Atom(-Function('pre', (action_, __t))))
                    precondition_2_body = (*effect_rule.body, *conditions, precondition)
                    precondition_2 = NormalRule(precondition_2_head, precondition_2_body)
                    diagnostic_domain_rules.append(precondition_2)

                for disables_rule in disables_p.rules:
                    if disables_rule.is_rule() and disables_rule.is_normal_rule():
                        robot_components = disables_rule.head.atom.symbol.arguments[0].arguments
                        action__ = disables_rule.head.atom.symbol.arguments[1]
                        if action_ != action__:
                            break
                        conditions_ = tuple(
                            BasicLiteral(Atom(condition.arguments_append(__t))).strong_neg_to_default_neg() for
                            condition in
                            disables_rule.head.atom.symbol.arguments[2].arguments)
                        if conditions != conditions_:
                            break
                        broken = (
                        BasicLiteral(Atom(-Function('broken', robot_component.arguments).arguments_append(__t))) for
                        robot_component in robot_components)
                        broken_rule_head = effect
                        broken_rule_body = (*effect_rule.body,
                                            *conditions,
                                            *broken
                                            )
                        broken_rule = NormalRule(broken_rule_head, broken_rule_body)
                        diagnostic_domain_rules.append(broken_rule)

diagnostic_domain = Program(name='diagnostic_domain', parameters=(__t,), rules=diagnostic_domain_rules)
print(diagnostic_domain.custom_str('\n'))

#program diagnostic_domain(__t).
pre(move(R,P2),__t) :- robot(R), adjacent(P1,P2), not -pre(move(R,P2),__t).
-pre(move(R,P2),__t) :- robot(R), adjacent(P1,P2), at(R,P1,__t), at(_,P2,__t).
at(R,P2,__t+1) :- robot(R), adjacent(P1,P2), at(R,P1,__t), -broken(R,base,__t).


In [879]:
diagnostic_query = (expected, robot_components_str, disables_str, broken_str, diagnosis_str, diagnostic_domain)

In [880]:
list(evaluate_forwards(diagnostic_query,
                       parts=(
                           ('base', ()),
                           ('expected', ()),
                           ('broken', ()),
                           *(('broken', (clingo.Number(n),)) for n in range(0, first_discrepancy.constant.number)),
                           *(('diagnosis', (clingo.Number(n),)) for n in range(0, first_discrepancy.constant.number)),
                           *(('diagnostic_domain', (clingo.Number(n),)) for n in
                             range(0, first_discrepancy.constant.number)),
                       )
                       , report=True));

UNSAT

<block>:1:27-48: info: atom does not occur in any rule head:
  move(charger,(2,3),0)

<block>:1:57-77: info: atom does not occur in any rule head:
  move(worker,(2,1),0)

<block>:7:28-36: info: atom does not occur in any rule head:
  robot(R)

<block>:13:49-57: info: atom does not occur in any rule head:
  robot(R)

<block>:13:59-74: info: atom does not occur in any rule head:
  adjacent(P1,P2)

<block>:42:57-65: info: atom does not occur in any rule head:
  robot(R)

<block>:42:67-82: info: atom does not occur in any rule head:
  adjacent(P1,P2)

<block>:42:134-142: info: atom does not occur in any rule head:
  robot(R)

<block>:42:144-159: info: atom does not occur in any rule head:
  adjacent(P1,P2)

<block>:42:207-215: info: atom does not occur in any rule head:
  robot(R)

<block>:42:217-232: info: atom does not occur in any rule head:
  adjacent(P1,P2)

